# Setup notebook for Apache Nifi ETL flow

## Config code - not working (for outdated version of NIFI)

create dirs for nifi custom config data

In [5]:
!mkdir -p nifi-conf nifi-state nifi-flowfile-repo nifi-content-repo nifi-provenance-repo nifi-database-repo

Config nifi users (admin)

In [3]:
%%writefile ./nifi-conf/nifi-users.xml

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<users>
  <!-- Define the admin user -->
  <user>
    <identifier>admin</identifier>
    <identity>admin</identity>
    <justCreated>false</justCreated>
  </user>

  <!-- Define policies for the admin user -->
  <policies>
    <!-- Access the controller -->
    <policy>
      <resource>/flow</resource>
      <action>read</action>
      <users>
        <user>admin</user>
      </users>
    </policy>
    <policy>
      <resource>/flow</resource>
      <action>write</action>
      <users>
        <user>admin</user>
      </users>
    </policy>
    <policy>
      <resource>/flow</resource>
      <action>delete</action>
      <users>
        <user>admin</user>
      </users>
    </policy>

    <!-- Access to resources and services -->
    <policy>
      <resource>/provenance</resource>
      <action>read</action>
      <users>
        <user>admin</user>
      </users>
    </policy>
    <policy>
      <resource>/controller</resource>
      <action>read</action>
      <users>
        <user>admin</user>
      </users>
    </policy>
    <policy>
      <resource>/controller</resource>
      <action>write</action>
      <users>
        <user>admin</user>
      </users>
    </policy>
    <policy>
      <resource>/counters</resource>
      <action>read</action>
      <users>
        <user>admin</user>
      </users>
    </policy>
    <policy>
      <resource>/policies</resource>
      <action>write</action>
      <users>
        <user>admin</user>
      </users>
    </policy>
  </policies>
</users>


Writing ./nifi-conf/nifi-users.xml


Config nifi files which enable authorisation

In [4]:
%%writefile ./nifi-conf/nifi.properties


#No HTTPS - DONT DO IN PROD
# nifi.web.https.host=0.0.0.0
# nifi.web.https.port=8443


# Web properties
nifi.web.http.host=
nifi.web.http.port=8080
nifi.web.http.network.interface.default=

# Security properties
nifi.security.user.login.identity.provider=single-user-provider
nifi.security.user.single.user=admin
nifi.security.user.single.password=admin
nifi.security.user.authorizer=single-user-authorizer

# Authorizer Configuration
nifi.security.user.authorizer=managed-authorizer
nifi.authorizer.configuration.file=./conf/authorizers.xml

# File-based user group provider
nifi.security.user.login.identity.provider.configuration.file=./conf/login-identity-providers.xml

# Other properties can be left as default or configured as per your requirement
# Configure the log directory
nifi.logging.dir=/opt/nifi/current/logs

# Specify the path for nifi-app.log
nifi-app.log=/opt/nifi/current/logs/nifi-app.log

nifi.nar.library.directory=./lib/nar



Overwriting ./nifi-conf/nifi.properties


Create additional files necesarry for nifi.properties

In [5]:
%%writefile ./nifi-conf/authorizers.xml
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<authorizers>
    <authorizer>
        <identifier>managed-authorizer</identifier>
        <class>org.apache.nifi.authorization.FileAuthorizer</class>
        <property name="Authorizations File">./conf/authorizations.xml</property>
        <property name="Users File">./conf/nifi-users.xml</property>
        <property name="Initial Admin Identity">admin</property>
        <property name="Legacy Authorized Users File"></property>
        <property name="Node Identity 1"></property>
    </authorizer>
</authorizers>


Overwriting ./nifi-conf/authorizers.xml


In [6]:
%%writefile ./nifi-conf/login-identity-providers.xml
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<loginIdentityProviders>
    <provider>
        <identifier>single-user-provider</identifier>
        <class>org.apache.nifi.authentication.SingleUserLoginIdentityProvider</class>
    </provider>
</loginIdentityProviders>


Overwriting ./nifi-conf/login-identity-providers.xml


Create bootstrap.conf for NIFI

In [7]:
%%writefile ./nifi-conf/bootstrap.conf
# Specify the fully qualified Java command to run
java=/usr/bin/java

# Specify the lib directory
lib.dir=./lib

# Specify the conf directory
conf.dir=./conf

# Specify the graceful shutdown timeout in seconds
graceful.shutdown.seconds=30

# JVM arguments
java.arg.1=-Xms512m
java.arg.2=-Xmx1024m
java.arg.3=-Xlog:gc
java.arg.4=-Xlog:gc*:file=/opt/nifi/current/logs/gc.log




Overwriting ./nifi-conf/bootstrap.conf


Create file for NIFI logging

In [8]:
%%writefile ./nifi-conf/logback.xml
<configuration>

  <!-- Console appender -->
  <appender name="CONSOLE" class="ch.qos.logback.core.ConsoleAppender">
    <encoder>
      <pattern>%d{ISO8601} %-5level [%thread] %logger{36} - %msg%n</pattern>
    </encoder>
  </appender>

  <!-- File appender for NiFi application logs -->
  <appender name="FILE" class="ch.qos.logback.core.rolling.RollingFileAppender">
    <file>/opt/nifi/nifi-current/logs/nifi-app.log</file>
    <rollingPolicy class="ch.qos.logback.core.rolling.TimeBasedRollingPolicy">
      <fileNamePattern>/opt/nifi/nifi-current/logs/nifi-app-%d{yyyy-MM-dd}.%i.log</fileNamePattern>
      <timeBasedFileNamingAndTriggeringPolicy class="ch.qos.logback.core.rolling.SizeAndTimeBasedFNATP">
        <maxFileSize>100MB</maxFileSize>
      </timeBasedFileNamingAndTriggeringPolicy>
      <maxHistory>30</maxHistory>
    </rollingPolicy>
    <encoder>
      <pattern>%d{ISO8601} %-5level [%thread] %logger{36} - %msg%n</pattern>
    </encoder>
  </appender>

  <!-- Root logger configuration -->
  <root level="INFO">
    <appender-ref ref="CONSOLE"/>
    <appender-ref ref="FILE"/>
  </root>

</configuration>


Writing ./nifi-conf/logback.xml


Dockerfile for NIFI generation

In [9]:
%%writefile Dockerfile

FROM apache/nifi:latest


COPY ./ApacheNifiETL/nifi-conf/nifi.properties /opt/nifi/current/conf/
COPY ./ApacheNifiETL/nifi-conf/nifi-users.xml /opt/nifi/current/conf/
COPY ./ApacheNifiETL/nifi-conf/login-identity-providers.xml /opt/nifi/current/conf/
COPY ./ApacheNifiETL/nifi-conf/authorizers.xml /opt/nifi/current/conf/

Overwriting Dockerfile


## Yaml

.yaml file creation

In [1]:
%%writefile ../apache-nifi.yml
version: "3"

services:
  nifi:
    image: apache/nifi:latest
    ports:
      - "8443:8443" #server runs here on https
      - "8181:8181"
      - "10000:10000"
    volumes:
      - conf:/opt/nifi/current/conf
      - content:/opt/nifi/current/content_repository
      - db:/opt/nifi/current/database_repository
      - flowfile:/opt/nifi/current/flowfile_repository
      - provenance:/opt/nifi/current/provenance_repository
      - logs:/opt/nifi/current/logs
      - data:/opt/nifi/current/data

    

    restart:
      always

    depends_on:
        clickhouse:
          condition: service_healthy
        mongo_manager_db_service:
          condition: service_healthy
        mongo_user_db_service:
          condition: service_healthy
        park_transactions_db:
          condition: service_healthy


      

volumes:
  conf:
  content:
  db:
  flowfile:
  provenance:
  logs:
  data:



Overwriting ../apache-nifi.yml


After bootup, wait about 30 seconds for server to start. NIFI creates random username and password when container deploys. You can then connect to https://localhost/8443/nifi (ignore certificate warning # TODO: implement certificates in PROD). Use docker logs for container and search temp user login and password values in log. 